In [2]:
import cheffrey
import pandas as pd

In [3]:
recipe_df = pd.read_csv('../data/recipes.csv')
reviews_df = pd.read_csv('../data/reviews.csv')

In [9]:
#there's so many recipes, I only want the best
review_summary = reviews_df.groupby('RecipeId').agg(
    nReviews=('ReviewId', 'size'),
    medianRating=('Rating', 'median'),
    percFiveStars=('Rating', lambda x: (x == 5).mean())
).reset_index()

In [16]:
rel_recipes = review_summary[review_summary['nReviews'] > 100]
print(rel_recipes.shape)
rel_recipes = rel_recipes[review_summary['percFiveStars'] > .80]
print(rel_recipes.shape)
rel_uids = rel_recipes['RecipeId']



(1042, 4)
(400, 4)


/var/folders/br/hdgw7t_11777tltdjs9tcsnc0000gn/T/ipykernel_19357/3127027427.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rel_recipes = rel_recipes[review_summary['percFiveStars'] > .80]


In [19]:
rel_recipes = recipe_df[recipe_df['RecipeId'].isin(rel_uids)]

In [107]:
def get_image(x):
    return x.split('"')[1]

def get_list(x):
    x = x.replace('c(', '').replace(')', '')
    x = x.replace('"', '')
    return x
rel_recipes['title'] = rel_recipes['Name']
rel_recipes['author'] = rel_recipes['AuthorName']
rel_recipes['total_time'] = rel_recipes['TotalTime'].str.replace('PT', '')
rel_recipes['yields'] = rel_recipes['RecipeYield']
rel_recipes['image'] = rel_recipes['Images'].apply(get_image)
rel_recipes['instructions'] = rel_recipes['RecipeInstructions'].apply(get_list)
rel_recipes['category'] = rel_recipes['RecipeCategory'] + ' ' + rel_recipes['Keywords'].fillna('').apply(get_list)

quantities = rel_recipes['RecipeIngredientQuantities'].apply(lambda x: get_list(x.replace('NA', '')).split(',')).tolist()
ingredients = rel_recipes['RecipeIngredientParts'].apply(lambda x: get_list(x.replace('NA', '')).split(',')).tolist()

out = []
for r_q, r_i in zip(quantities, ingredients):
    l = []
    for q, i in zip(r_q, r_i):
        l += [q + ' '+ i]
    out.append(l)

rel_recipes['ingredients'] = out

/var/folders/br/hdgw7t_11777tltdjs9tcsnc0000gn/T/ipykernel_19357/792388921.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_recipes['title'] = rel_recipes['Name']
/var/folders/br/hdgw7t_11777tltdjs9tcsnc0000gn/T/ipykernel_19357/792388921.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_recipes['author'] = rel_recipes['AuthorName']
/var/folders/br/hdgw7t_11777tltdjs9tcsnc0000gn/T/ipykernel_19357/792388921.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [112]:
current_recipes = cheffrey.load_local_recipes()

tags = list(set([key for x in current_recipes.values() for key in x]))
print(len(current_recipes))

1661


In [109]:
for i, row in rel_recipes.iterrows():
    name = row['Name']
    if name in current_recipes:
        name += ' (Kaggle)'
    
    current_recipes[name] = {
        'title': row['title'], 
        'author': row['author'], 
        'total_time': row['total_time'], 
        'yields': row['yields'], 
        'image': row['image'], 
        'instructions': row['instructions'], 
        'category': row['category'], 
        'ingredients': row['ingredients'],
        'source': 'kaggle',
        'url': None
    }


In [110]:
print(len(current_recipes))

1661


In [111]:
import json
with open(cheffrey.ROOT_DIR/'data/recipes.json', 'w') as f:
    json.dump(current_recipes, f)